In [6]:
pip install --upgrade plotly

In [7]:
pip install -U kaleido

### Import Statements

In [8]:
import pandas as pd
import plotly.graph_objects as go
import PIL
import io

### Notebook Presentation

In [9]:
pd.options.display.float_format = '{:,.2f}'.format

### Read the Data

In [10]:
df = pd.read_csv('covid_data.csv')

In [11]:
df.head()


,S. No.,Country Name,Cases,Deaths,Recovered,Death_Rate,ISO
0,1.00,Peru,4524748,220831,4303917,4.90,PER
1,2.00,Bulgaria,1329266,38700,1290566,2.90,BGR
2,3.00,Bosnia and Herzegovina,403638,16388,387250,4.10,BIH
3,4.00,Hungary,2230381,49051,2181330,2.20,HUN
4,5.00,North Macedonia,350589,9977,340612,2.80,MKD


### Data preparation

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   S. No.        229 non-null    float64
 1   Country Name  229 non-null    object 
 2   Cases         229 non-null    int64  
 3   Deaths        229 non-null    int64  
 4   Recovered     229 non-null    int64  
 5   Death_Rate    229 non-null    float64
 6   ISO           229 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 12.6+ KB


In [19]:
import plotly.express as px
from plotly import colors


fig = go.Figure()

# Create a custom color scale
custom_colorscale = [
    [0, '#efdbe0'],
    [0.5, '#c20110'],
    [1, '#550001']
]

# Frames list for smooth opacity transition between Death Rate and Cases
frames = []
opacity_values = [num/39 for num in range(1, 40)] + [1]*40 + [num/39 for num in range(39, 0, -1)]


# Create the figure with the initial data
fig.add_trace(go.Choropleth(
              locations=df["ISO"],
              z=df["Cases"],
              text=df["Country Name"],
              colorscale=custom_colorscale,
              colorbar_title="Cases",
              hovertemplate="Country: %{text}<br>Death Rate: %{z}",
              marker_opacity=0.066,
        )
    )


# Add the frame for the Cases map with opacity fade-in
for i, alpha in enumerate(opacity_values):
    frames.append(go.Frame(
        data=[
            go.Choropleth(
                locations=df["ISO"],
                z=df["Cases"],
                text=df["Country Name"],
                colorscale=custom_colorscale,
                colorbar_title="Cases",
                hovertemplate="Country: %{text}<br>Cases: %{z}",
                marker_opacity=alpha,
                )
            ],
         layout=go.Layout(title_text="Total COVID-19 Cases Worldwide")
    ))


# Create transition frames for opacity from Death Rate to Cases
for i, alpha in enumerate(opacity_values):
    frames.append(go.Frame(
        data=[
            go.Choropleth(
                locations=df["ISO"],
                z=df["Death_Rate"],
                text=df["Country Name"],
                colorscale=custom_colorscale,
                hovertemplate="Country: %{text}<br>Death Rate: %{z}",
                marker_opacity=alpha,
                colorbar_title="Death Rate, %"
            )
        ],
        layout=go.Layout(title_text="COVID-19 Death Rate (%)")
    ))



# Add frames to the figure
fig.frames = frames

# Update layout with updated title text and smooth transition
fig.update_layout(
    title={
        'y': 0.78,
        'x': 0.55,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    mapbox_style="carto-positron",
    title_font_size=34,
    height=1000,
    width=1200,
    paper_bgcolor='white',
    plot_bgcolor='white',
    # updatemenus=[
    #     {
    #         'buttons': [
    #             {
    #                 'args': [None, {'frame': {'duration': 40, 'redraw': True}, 'fromcurrent': True}],
    #                 'label': 'Play',
    #                 'method': 'animate'
    #             },
    #             {
    #                 'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}],
    #                 'label': 'Pause',
    #                 'method': 'animate'
    #             }
    #         ],
    #         'direction': 'left',
    #         'pad': {'r': 5, 't': 20},
    #         'showactive': False,
    #         'type': 'buttons',
    #         'x': 0.1,
    #         'xanchor': 'right',
    #         'y': 0,
    #         'yanchor': 'top'
    #     }
    # ],
    # sliders=[{
    #     'active': 0,
    #     'yanchor': 'top',
    #     'xanchor': 'left',
    #     'currentvalue': {
    #         'font': {'size': 20},
    #         'prefix': 'View: ',
    #         'visible': True,
    #         'xanchor': 'right'
    #     },
    #     'transition': {'duration': 100, 'easing': 'cubic-in-out'},
    #     'pad': {'b': 10, 't': 50},
    #     'len': 0.9,
    #     'x': 0.1,
    #     'y': 0,
    #     'steps': [
    #         {'args': [[f.name], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}], 'label': f.name, 'method': 'animate'}
    #         for f in frames
    #     ]
    # }]
)
fig.update_geos(projection_type="natural earth",
                showcoastlines=True, coastlinecolor="Black",
                showland=True, landcolor="white"
                )


fig.update_traces(colorbar=dict(
                      len=0.6,
                      thickness=40,
                      x=-0.2,
                      xanchor='left',
                      title=dict(side='right', font=dict(size=26)),
                      tickfont=dict(size=18))
                  )

# Show the figure
fig.show()

### Saving the Chart in GIF format

In [20]:
frames_img = []
for s, fr in enumerate(fig.frames):
    # Update the figure with the current frame data
    fig.update(data=fr.data)
    # Ensure the correct layout is applied for each frame
    fig.update_layout(fr.layout)

    # Capture the image of the current frame
    img_bytes = fig.to_image(format="png")
    img = PIL.Image.open(io.BytesIO(img_bytes))
    frames_img.append(img)



# create animated GIF
frames_img[0].save(
    "test3.gif",
    save_all=True,
    append_images=frames_img[1:],
    optimize=False,
    duration=40,
    loop=0,
)